In [ ]:
import ROOT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import math
import uproot
from array import array

In [ ]:
#data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndhist_data.root"

# First attempt to fix angular coordinates
#data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndhist_widths_coord_data.root"

data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndhist_widths_data_xf.root"

#mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndhist_mc.root"

mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndwidths_mc_xf.root"
print("Got MC and Data Files to compare and analyze")

In [ ]:
rfile_data = ROOT.TFile.Open(data_file)
rfile_mc = ROOT.TFile.Open(mc_file)

DATA_TAG = "v10_06_00_02"

rfile_data.ls()

# Define Iterative Truncated Mean Function

In [ ]:
def iterative_truncated_mean(h, sig_down, sig_up, tol):
    #initialize the final result
    result_mean = 0
    result_std = 0

    mean = h.GetMean()
    std_dev = h.GetRMS()
    if ROOT.TMath.Abs(result_mean - mean) < tol:
        return [result_mean, result_std]
    
    result_mean = mean
    result_std = std_dev

    flag=True
    h_prev = h.Clone("h_prev")
    i=1
    while flag:
        quantile_fractions = array('d', [0.5])
        quantile_values = array('d', [0.0]*len(quantile_fractions))
        h_prev.GetQuantiles(len(quantile_fractions), quantile_values, quantile_fractions)

        median = quantile_values[0]
        h_new = h_prev.Clone("h_new")
        h_new.Reset()
        for bin in range(1, h_prev.GetNbinsX() + 1):
            if h_prev.GetBinLowEdge(bin) > median + sig_up * std_dev or h_prev.GetBinLowEdge(bin) +h_prev.GetBinWidth(bin) < median - sig_down * std_dev:
                continue
            h_new.SetBinContent(bin, h_prev.GetBinContent(bin))
            h_new.SetBinError(bin, h_prev.GetBinError(bin))
        
        mean = h_new.GetMean()
        std_dev = h_new.GetRMS()
        if ROOT.TMath.Abs(result_mean - mean) < tol:
            flag = False
            return [result_mean, result_std]
        
        result_mean = mean
        result_std = std_dev
        h_prev.SetDirectory(0)
        h_prev.Delete()
        h_prev = h_new.Clone("h_prev")
        i += 1
        if i > 100:
            print("Warning: Iteration limit reached, returning last computed mean and std_dev")
            return [result_mean, result_std]
        
print("Iterative truncated mean function defined")

# Define Profiling Function

In [ ]:
# Function to profile a 2D histogram along X axis and compute iterative truncated mean for each slice

def profile_2d_histogram(file_name, dim, tpc, plane):
    means = []
    errors = []
    idx = 3*tpc+plane
    rfile = ROOT.TFile.Open(file_name)
    h = rfile.Get("hwidth"+str(idx))

    # Loop over the bins in the selected dimension
    for bin in range(1, h.GetAxis(dim).GetNbins() + 1):
        h.GetAxis(dim).SetRange(bin, bin)
        h_1d_temp = h.Projection(6)
        result = iterative_truncated_mean(h_1d_temp, 2, 1.75, 1e-4)
        means.append(result[0])
        h_1d_temp.Delete()
        #errors.append(result[1]/math.sqrt(h_1d_temp.GetEntries()))
        errors.append(result[1])
    
    return means, errors

print("Profiling function defined")                  

In [ ]:
# I want to convert a ROOT TH2D to a plt.hist2d for better plotting
def root_th2d_to_plt_hist2d(th2d):
    # Get the bin edges and contents from the ROOT TH2D
    x_edges = [th2d.GetXaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsX() + 2)]
    y_edges = [th2d.GetYaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsY() + 2)]
    z_values = np.zeros((th2d.GetNbinsX(), th2d.GetNbinsY()))

    for i in range(1, th2d.GetNbinsX() + 1):
        for j in range(1, th2d.GetNbinsY() + 1):
            z_values[i-1, j-1] = th2d.GetBinContent(i, j)

    return np.array(x_edges), np.array(y_edges), np.array(z_values)

# Example usage:
# th2d = rfile_data.Get("hwidth0")
# x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
# plt.hist2d(x_edges[:-1], y_edges[:-1], z_values.T, bins=[x_edges, y_edges], cmap='viridis')
# plt.colorbar(label='Counts')
# plt.xlabel('X-axis label')                
#plt.ylabel('Y-axis label')
# plt.title('2D Histogram from ROOT TH2D')
# plt.show()    

# I want to convert a ROOT TH1D with errors to a plt.errorbar for better plotting
def root_th1d_to_plt_errorbar(th1d):
    # Get the bin centers, contents, and errors from the ROOT TH1D
    x_values = [th1d.GetBinCenter(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_values = [th1d.GetBinContent(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_errors = [th1d.GetBinError(i) for i in range(1, th1d.GetNbinsX() + 1)]

    return np.array(x_values), np.array(y_values), np.array(y_errors) 

# Example usage:
# th1d = rfile_data.Get("hwidth0").ProjectionX()
# x_values, y_values, y_errors = root_th1d_to_plt_errorbar(th1d)
# plt.errorbar(x_values, y_values, yerr=y_errors, fmt='o', ecolor='red', capsize=5)
# plt.xlabel('X-axis label')                
# plt.ylabel('Y-axis label')
# plt.title('1D Histogram with Error Bars from ROOT TH1D')
# plt.show()


# Function to create a ROOT TH1D from means and errors
def make_mean_hist(h2d, means, errors):
    h_means = h2d.ProjectionX()
    h_means.Reset()
    for i in range(1, h_means.GetNbinsX() + 1):
        h_means.SetBinContent(i, means[i-1])
        h_means.SetBinError(i, errors[i-1])
    
    return h_means

In [ ]:
dim_dict = {0: "x", 1: "y", 2: "z", 3:"txz", 4:"tyz", 5:"dqdx"}
label_dict = {0: "Reconstructed Hit X [cm]", 
              1: "Reconstructed Hit Y [cm]", 
              2: "Reconstructed Hit Z [cm]", 
              3:"Reconstructed Hit "+r"$\theta_{XZ}$ [degrees]", 
              4:"Reconstructed Hit "+r"$\theta_{YZ}$ [degrees]", 
              5:"Reconstructed Hit dQ/dx [ADC/cm]"}

In [ ]:

def plot_profiles_with_fits(DIM, isData=True, xlim0=None, xlim1=None, zlim=None, DATA_TAG=None):

    kNtpcs = 2
    kNplanes = 3



    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    gs = gridspec.GridSpec(1, 2 + 1, width_ratios=[1, 1, 0.05], wspace=0.1)

    largest_max_z = 0

    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane
            #thn = rfile_data.Get("hwidth"+str(idx))
            h = 0
            if isData:
                h = rfile_data.Get("hwidth"+str(idx))
            else:
                h = rfile_mc.Get("hwidth"+str(idx))

            h.GetAxis(DIM).SetRange(1, h.GetAxis(DIM).GetNbins()+1)
            th2d = h.Projection(6, DIM)
            #th2d = rfile_data.Get("hntrk_"+str(idx)+"_"+dim_dict[DIM])
            means, errors = profile_2d_histogram(data_file, DIM, tpc, plane)
            #print("mean of bin 30", means[30])
            h_means = make_mean_hist(th2d, means, errors)

            x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
            z_masked = np.ma.masked_where(z_values == 0, z_values)

            cmap = plt.cm.viridis.copy()
            cmap.set_bad(color='white')

            ax = axs[tpc, plane]
            X, Y = np.meshgrid(x_edges, y_edges, indexing='ij')  # Needed for pcolormesh
            #ax.pcolormesh(X, Y, z_values, shading='auto', cmap='viridis')
            MAX_Z = np.max(z_values)
            if MAX_Z > largest_max_z:
                largest_max_z = MAX_Z
            if zlim is not None:
                pcm = ax.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto', norm=LogNorm(vmin=zlim[0], vmax=zlim[1]))
            else:
                pcm = ax.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto', norm=LogNorm(vmin=1, vmax=MAX_Z))

            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)
                
            #im = ax.hist2d(x_edges, y_edges, z_values.T, cmap='viridis')
            ax.set_title(f'TPC {tpc}, Plane {plane}')
            #fig.colorbar(im[3], ax=ax, label='')

            x, y, yerr = root_th1d_to_plt_errorbar(h_means)
            ax.errorbar(x, y, xerr=np.ones_like(yerr)*((x_edges[1]-x_edges[0])/2), 
                        yerr=yerr, fmt='o', ecolor='red', capsize=0, label='Iterative Truncated Mean', markersize=0)
            h.Delete()
            th2d.Delete()
            h_means.Delete()
    print("Largest max z value:", largest_max_z)
    fig.supxlabel(label_dict[DIM], fontsize=20)
    fig.supylabel("Reconstructed Hit Width [ticks]", fontsize=20)
    t = "SBND Off-Beam "
    if isData:
        t += "Data: "
    else:
        t += "MC: "
    if DATA_TAG is not None:
        t += DATA_TAG
    fig.suptitle(t, fontsize=20)
    cbar_ax = fig.add_subplot(gs[0, 2])
    fig.colorbar(pcm, cax=cbar_ax)
    #fig.colorbar(pcm, ax=axs, fraction=.05, label='')
    plt.tight_layout()
    plt.show()

print("Defining plot function")

In [ ]:
plot_profiles_with_fits(0, isData=True, xlim0=[-220, 20], xlim1=[-20, 220], zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(0, isData=False, xlim0=[-220, 20], xlim1=[-20, 220], zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(1, isData=True, xlim0=[-220, 220], xlim1=[-220, 220], zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(1, isData=False, xlim0=[-220, 220], xlim1=[-220, 220], zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(2, isData=True, xlim0=[-20, 520], xlim1=[-20, 520], zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(2, isData=False, xlim0=[-20, 520], xlim1=[-20, 520], zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(3, isData=True, xlim0=[-100, 100], xlim1=[-100, 100], zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(3, isData=False, xlim0=[-100, 100], xlim1=[-100, 100], zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(4, isData=True, xlim0=[-100, 100], xlim1=[-100, 100], zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(4, isData=False, xlim0=[-100, 100], xlim1=[-100, 100], zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(5, isData=True, xlim0=None, xlim1=None, zlim=[1, 10**5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profiles_with_fits(5, isData=False, xlim0=None, xlim1=None, zlim=[1, 10**5], DATA_TAG=DATA_TAG)

# Make Ratios

In [ ]:

def plot_profile_ratios(DIM, xlim0=None, xlim1=None, ylim=None, DATA_TAG=None):

    kNtpcs = 2
    kNplanes = 3

    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    


    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane
            #thn = rfile_data.Get("hwidth"+str(idx))
            
            # Get THn histos
            h_data = rfile_data.Get("hwidth"+str(idx))
            h_mc = rfile_mc.Get("hwidth"+str(idx))

            h_data.GetAxis(DIM).SetRange(1, h_data.GetAxis(DIM).GetNbins()+1)
            h_mc.GetAxis(DIM).SetRange(1, h_mc.GetAxis(DIM).GetNbins()+1)

            th2d_data = h_data.Projection(6, DIM)
            th2d_mc = h_mc.Projection(6, DIM)

           
            means_data, errors_data = profile_2d_histogram(data_file, DIM, tpc, plane)
            means_data = np.array(means_data)
            
            means_mc, errors_mc = profile_2d_histogram(mc_file, DIM, tpc, plane)
            means_mc = np.array(means_mc)

            #print("mean of bin 30", means[30])
            h_means_data = make_mean_hist(th2d_data, means_data, errors_data)
            bin_centers = []
            for bin in range(1, h_means_data.GetNbinsX() + 1):
                bin_centers.append(h_means_data.GetBinCenter(bin))
            
            ratios = means_data / means_mc
            ratio_errors = np.sqrt((errors_data / means_mc)**2 + (means_data * errors_mc / means_mc**2)**2)

            ax = axs[tpc, plane]

            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
                    ax.plot(xlim0, [1, 1], c="black", linestyle="--", linewidth=2)

            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)
                    ax.plot(xlim1, [1, 1], c="black", linestyle="--", linewidth=2)
            if ylim is not None:
                ax.set_ylim(ylim)
                
            #im = ax.hist2d(x_edges, y_edges, z_values.T, cmap='viridis')
            ax.set_title(f'TPC {tpc}, Plane {plane}')
            #fig.colorbar(im[3], ax=ax, label='')

            ax.errorbar(bin_centers, ratios, xerr=np.ones_like(bin_centers)*((bin_centers[1]-bin_centers[0])/2), 
                        yerr=ratio_errors, fmt='o', ecolor='red', capsize=0, label='Iterative Truncated Mean', markersize=0)
            h_data.Delete()
            h_mc.Delete()
            th2d_data.Delete()
            th2d_mc.Delete()
            h_means_data.Delete()
            
    
    fig.supxlabel(label_dict[DIM], fontsize=20)
    fig.supylabel("Data / Simulation", fontsize=20)
    t = "SBND Off-Beam: "

    if DATA_TAG is not None:
        t += DATA_TAG
    fig.suptitle(t, fontsize=20)
    

    plt.tight_layout()
    plt.show()

print("Defining plot function")

In [ ]:
plot_profile_ratios(0, xlim0=[-220, 20], xlim1=[-20, 220], ylim=[0.5, 1.5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profile_ratios(3, xlim0=[-100, 100], xlim1=[-100, 100], ylim=[0.5, 1.5], DATA_TAG=DATA_TAG)

In [ ]:
plot_profile_ratios(5, xlim0=[0, 5000], xlim1=[0, 5000], ylim=[0.5, 1.5], DATA_TAG=DATA_TAG)

# Ratios with Splines

In [ ]:
def plot_profile_ratios_splines(DIM, xlim0=None, xlim1=None, ylim=None, DATA_TAG=None):

    kNtpcs = 2
    kNplanes = 3

    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    


    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane
            #thn = rfile_data.Get("hwidth"+str(idx))
            
            # Get THn histos
            h_data = rfile_data.Get("hwidth"+str(idx))
            h_mc = rfile_mc.Get("hwidth"+str(idx))

            h_data.GetAxis(DIM).SetRange(1, h_data.GetAxis(DIM).GetNbins()+1)
            h_mc.GetAxis(DIM).SetRange(1, h_mc.GetAxis(DIM).GetNbins()+1)

            th2d_data = h_data.Projection(6, DIM)
            th2d_mc = h_mc.Projection(6, DIM)

           
            means_data, errors_data = profile_2d_histogram(data_file, DIM, tpc, plane)
            means_data = np.array(means_data)
            
            means_mc, errors_mc = profile_2d_histogram(mc_file, DIM, tpc, plane)
            means_mc = np.array(means_mc)

            #print("mean of bin 30", means[30])
            h_means_data = make_mean_hist(th2d_data, means_data, errors_data)
            bin_centers = []
            for bin in range(1, h_means_data.GetNbinsX() + 1):
                bin_centers.append(h_means_data.GetBinCenter(bin))
            bin_centers = np.array(bin_centers)
            ratios = means_data / means_mc
            ratio_errors = np.sqrt((errors_data / means_mc)**2 + (means_data * errors_mc / means_mc**2)**2)
            spline_ratios = None
            x_vals = None
            # Fit a spline to the ratios in the region of non-zero values
            non_zero_indices = np.where(ratios > 0)[0]
            if len(non_zero_indices) > 0:
                from scipy.interpolate import UnivariateSpline
                spline = UnivariateSpline(bin_centers[non_zero_indices], ratios[non_zero_indices], s=0.01)
                w = bin_centers[1] - bin_centers[0]
                x_vals = np.linspace(bin_centers[0], bin_centers[-1], 1000)
                spline_ratios = spline(x_vals)
                #ratio_errors = np.sqrt((errors_data / means_mc)**2 + (means_data * errors_mc / means_mc**2)**2)

            ax = axs[tpc, plane]

            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
                    ax.plot(xlim0, [1, 1], c="black", linestyle="--", linewidth=2)

            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)
                    ax.plot(xlim1, [1, 1], c="black", linestyle="--", linewidth=2)
            if ylim is not None:
                ax.set_ylim(ylim)
                
            #im = ax.hist2d(x_edges, y_edges, z_values.T, cmap='viridis')
            ax.set_title(f'TPC {tpc}, Plane {plane}')
            #fig.colorbar(im[3], ax=ax, label='')

            ax.errorbar(bin_centers, ratios, xerr=np.ones_like(bin_centers)*((bin_centers[1]-bin_centers[0])/2), 
                        yerr=ratio_errors, fmt='o', ecolor='red', capsize=0, label='Iterative Truncated Mean', markersize=0)
            
            ax.plot(x_vals, spline_ratios, c='blue', label='Spline Fit', linewidth=2, linestyle='--')

            ax.legend()
            h_data.Delete()
            h_mc.Delete()
            th2d_data.Delete()
            th2d_mc.Delete()
            h_means_data.Delete()
            
    
    fig.supxlabel(label_dict[DIM], fontsize=20)
    fig.supylabel("Data / Simulation", fontsize=20)
    t = "SBND Off-Beam: "

    if DATA_TAG is not None:
        t += DATA_TAG
    fig.suptitle(t, fontsize=20)
    

    plt.tight_layout()
    plt.show()

print("Defining plot function")

In [ ]:
plot_profile_ratios_splines(5, xlim0=[0, 5000], xlim1=[0, 5000], ylim=[0.5, 1.5], DATA_TAG=DATA_TAG)

# Multidimensional Splines

In [ ]:
def profile_3d_histogram(file_name, dim1, dim2, tpc, plane):
    # dim1 is the x-axis dimension
    # dim2 is the y-axis dimension
    #means = []
    #errors = []
    idx = 3*tpc+plane
    rfile = ROOT.TFile.Open(file_name)
    h = rfile.Get("hwidth"+str(idx))
    h2d_means = h.Projection(dim2, dim1)
    h2d_means.SetDirectory(0)
    h2d_means.Reset()
    h2d_errors = h.Projection(dim2, dim1)
    h2d_errors.SetDirectory(0)
    h2d_errors.Reset()
    # Loop over the bins in the selected dimension
    for binx in range(1, h.GetAxis(dim1).GetNbins() + 1):
        for biny in range(1, h.GetAxis(dim2).GetNbins() + 1):
            h.GetAxis(dim1).SetRange(binx, binx)
            h.GetAxis(dim2).SetRange(biny, biny)
            h_1d_temp = h.Projection(6)
            result = iterative_truncated_mean(h_1d_temp, 2, 1.75, 1e-4)
            h2d_means.SetBinContent(binx, biny, result[0])
            h2d_errors.SetBinContent(binx, biny, result[1])
     
            h_1d_temp.Delete()
        
    
    return h2d_means, h2d_errors

print("3D Profiling function defined") 

In [ ]:

def plot_2D_fits(DIMX, DIMY, isData=True, xlim0=None, xlim1=None, ylim0=None, ylim1=None, zlim=None, DATA_TAG=None):

    kNtpcs = 2
    kNplanes = 3

    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    gs = gridspec.GridSpec(1, 2 + 1, width_ratios=[1, 1, 0.05], wspace=0.1)

    largest_max_z = 0

    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane
            #thn = rfile_data.Get("hwidth"+str(idx))
            h_means, h_errors = 0, 0
            if isData:
                h_means, h_errors = profile_3d_histogram(data_file, DIMX, DIMY, tpc, plane)
            else:
                h_means, h_errors = profile_3d_histogram(mc_file, DIMX, DIMY, tpc, plane)

            x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(h_means)
            z_masked = np.ma.masked_where(z_values == 0, z_values)

            cmap = plt.cm.viridis.copy()
            cmap.set_bad(color='white')

            ax = axs[tpc, plane]
            X, Y = np.meshgrid(x_edges, y_edges, indexing='ij')  # Needed for pcolormesh
            #ax.pcolormesh(X, Y, z_values, shading='auto', cmap='viridis')
            MAX_Z = np.max(z_values)
            if MAX_Z > largest_max_z:
                largest_max_z = MAX_Z
            if zlim is not None:
                pcm = ax.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto')
            else:
                pcm = ax.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto')

            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
                if ylim0 is not None:
                    ax.set_ylim(ylim0)
            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)
                if ylim1 is not None:
                    ax.set_ylim(ylim1)

                
            #im = ax.hist2d(x_edges, y_edges, z_values.T, cmap='viridis')
            ax.set_title(f'TPC {tpc}, Plane {plane}')
            #fig.colorbar(im[3], ax=ax, label='')
          
            h_means.Delete()
            h_errors.Delete()

    print("Largest max z value:", largest_max_z)
    fig.supxlabel(label_dict[DIMX], fontsize=20)
    fig.supylabel(label_dict[DIMY], fontsize=20)
    t = "SBND Off-Beam "
    if isData:
        t += "Data: "
    else:
        t += "MC: "
    if DATA_TAG is not None:
        t += DATA_TAG
    fig.suptitle(t, fontsize=20)
    cbar_ax = fig.add_subplot(gs[0, 2])
    fig.colorbar(pcm, cax=cbar_ax)
    #fig.colorbar(pcm, ax=axs, fraction=.05, label='')
    plt.tight_layout()
    plt.show()

print("Defining plot function")

In [ ]:
Stop